In [3]:
#import libraries
%matplotlib inline
from array import *
import time
import matplotlib
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import access_data as ad
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [4]:
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython: from IPython import display

In [5]:
#define the grid size of the environment
environment_rows = 17
environment_columns = 28

#define the q-table in 3 dimension for the row, column and 8 possible actions
#Initialize all the values in q-table to zero
q_values = np.zeros((environment_rows, environment_columns, 8))

In [7]:
#define actions
#numeric action codes: 0 = south, 1 = west, 2 = north, 3 = east, 4 = southwest, 5 = southeast, 6 = northwest, 7 =northeast
actions = ['S', 'W', 'N', 'E', 'SW', 'SE', 'NW', 'NE']

#input the rewards table from the experimentation data
rewards = []
with open('simple rewards 2.csv', 'r') as file:
    reader = pd.read_csv('simple rewards 2.csv', header=None, encoding = 'utf8', delimiter=',')
    for i in reader:
        rewards.append(round(reader[i], 2))
rewards = np.transpose(rewards)
pd.DataFrame(rewards)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,-2.0,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.0
1,-2.0,-2.00,-1.97,-1.93,-1.90,-1.86,-1.83,-1.79,-1.76,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-1.76,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-1.76,-1.82,-1.88,-1.94,-2.00,-2.0
2,-2.0,-2.00,-1.94,-1.87,-1.80,-1.74,-1.68,-1.61,-1.54,-1.48,-1.60,-1.64,-1.66,-1.67,-1.68,-1.48,-1.68,-1.67,-1.66,-1.64,-1.60,-1.48,-1.58,-1.69,-1.79,-1.90,-2.00,-2.0
3,-2.0,-2.00,-1.91,-1.82,-1.73,-1.64,-1.56,-1.47,-1.38,-1.29,-1.20,-1.28,-1.32,-1.34,-1.36,-1.20,-1.36,-1.34,-1.32,-1.28,-1.20,-1.33,-1.47,-1.60,-1.73,-1.87,-2.00,-2.0
4,-2.0,-2.00,-1.89,-1.78,-1.68,-1.57,-1.46,-1.35,-1.24,-1.14,-1.03,-0.92,-0.98,-1.02,-1.04,-0.92,-1.04,-1.02,-0.98,-0.92,-1.07,-1.23,-1.38,-1.54,-1.69,-1.85,-2.00,-2.0
5,-2.0,-2.00,-1.88,-1.75,-1.63,-1.51,-1.38,-1.26,-1.13,-1.01,-0.89,-0.76,-0.64,-0.69,-0.72,-0.64,-0.72,-0.69,-0.64,-0.81,-0.98,-1.15,-1.32,-1.49,-1.66,-1.83,-2.00,-2.0
6,-2.0,-2.00,-1.86,-1.73,-1.59,-1.45,-1.32,-1.18,-1.04,-0.91,-0.77,-0.63,-0.50,-0.36,-0.40,-0.36,-0.40,-0.36,-0.54,-0.72,-0.91,-1.09,-1.27,-1.45,-1.64,-1.82,-2.00,-2.0
7,-2.0,-2.00,-1.85,-1.70,-1.56,-1.41,-1.26,-1.11,-0.97,-0.82,-0.67,-0.52,-0.38,-0.23,-0.08,-0.08,-0.08,-0.27,-0.46,-0.66,-0.85,-1.04,-1.23,-1.42,-1.62,-1.81,-2.00,-2.0
8,-2.0,-1.86,-1.73,-1.60,-1.46,-1.32,-1.19,-1.06,-0.92,-0.78,-0.65,-0.52,-0.38,-0.24,-0.11,0.20,-0.02,-0.20,-0.38,-0.56,-0.74,-0.92,-1.10,-1.28,-1.46,-1.64,-1.82,-2.0
9,-2.0,-2.00,-1.85,-1.70,-1.56,-1.41,-1.26,-1.11,-0.97,-0.82,-0.67,-0.52,-0.38,-0.23,-0.08,-0.08,-0.08,-0.27,-0.46,-0.66,-0.85,-1.04,-1.23,-1.42,-1.62,-1.81,-2.00,-2.0


In [8]:
#define the function to calculate the decaying epsilon value
def EpsilonGreedyStrategy(epsilon, current_ep):
    start = epsilon[0]
    end = epsilon[1]
    decay = epsilon[2]
    rate = end + (start - end) * math.exp(-1. * current_ep * decay)
    return rate

In [9]:
#function to obtain the angle for the direction
def get_angle(s1, s2, s3, s4):
    list_value = [s1, s2, s3, s4]
    #Obtain highest and second highest value of the sensors
    mx=max(list_value[0],list_value[1])
    secondmax=min(list_value[0],list_value[1])
    n =len(list_value)
    for i in range(0,n):
        if list_value[i]>mx:
            secondmax=mx
            mx=list_value[i]
        elif list_value[i]>secondmax and \
            mx != list_value[i]:
            secondmax=list_value[i]

    max_index = list_value.index(mx)
    secondmax_index = list_value.index(secondmax)
    #Obtain the angle from two highest value of sensor in radian
    if list_value[max_index] == s1:
        if list_value[secondmax_index] == s2:
            angle = math.atan(list_value[secondmax_index]/list_value[max_index])
        elif list_value[secondmax_index] == s3:
            angle = math.radians(0) #east
        elif list_value[secondmax_index] == s4:
            angle = -math.atan(list_value[secondmax_index]/list_value[max_index])

    elif list_value[max_index] == s2:
        if list_value[secondmax_index] == s1:
            angle = math.atan(list_value[secondmax_index]/list_value[max_index])
        elif list_value[secondmax_index] == s3:
            angle = math.radians(180)-math.atan(list_value[max_index]/list_value[secondmax_index])
        elif list_value[secondmax_index] == s4:
            angle = math.radians(90) #north

    elif list_value[max_index] == s3:
        if list_value[secondmax_index] == s1:
            angle = math.radians(180) #west
        elif list_value[secondmax_index] == s2:
            angle = math.radians(180)-math.atan(list_value[secondmax_index]/list_value[max_index])
        elif list_value[secondmax_index] == s4:
            angle = -math.radians(180) + math.atan(list_value[secondmax_index]/list_value[max_index])

    elif list_value[max_index] == s4:
        if list_value[secondmax_index] == s1:
            angle = -math.atan(list_value[max_index]/list_value[secondmax_index])
        elif list_value[secondmax_index] == s2:
            angle = -math.radians(90) #south
        elif list_value[secondmax_index] == s3:
            angle = -math.radians(180) + math.atan(list_value[secondmax_index]/list_value[max_index])       
    return angle

#function to translate angle to direction
#Updated to quartile form
def transform_angle(theta):
    if theta > -math.radians(22.5) and theta < math.radians(22.5):
        direction = 'E'
    elif theta > math.radians(22.5) and theta < math.radians(67.5):
        direction = 'SE'
    elif theta > math.radians(67.5) and theta < math.radians(112.5):
        direction = 'S'
    elif theta > math.radians(112.5) and theta < math.radians(157.5):
        direction = 'SW'
    elif theta > math.radians(157.5) or theta < -math.radians(157.5):
        direction = 'W'
    elif theta > -math.radians(157.5) and theta < -math.radians(112.5):
        direction = 'NW'
    elif theta > -math.radians(112.5) and theta < -math.radians(67.5):
        direction = 'N'
    elif theta > -math.radians(67.5) and theta < -math.radians(22.5):
        direction = 'NE'
    return direction

In [10]:
#function to determine whether current state is terminal state
def is_terminal_state(current_row_index, current_column_index):
    #termination state if drone reached border or goal
    if rewards[current_row_index, current_column_index] == .2 or rewards[current_row_index, current_column_index] == -2:
        return True
    else:
        return False
    
#Terminal state for drone (source localization)
def is_terminator(current_row_index, current_column_index):
    #terminateion state if drone reached goal only
    if rewards[current_row_index, current_column_index] == .2:
        return True
    else:
        return False

#function to randomize non-terminal starting location
def get_starting_location():
    #get a random row and column index
    current_row_index = np.random.randint(environment_rows)
    current_column_index = np.random.randint(environment_columns)
    #continue choosing random row and column indexes until a non-terminal state is identified
    while is_terminal_state(current_row_index, current_column_index):
        current_row_index = np.random.randint(environment_rows)
        current_column_index = np.random.randint(environment_columns)
    return current_row_index, current_column_index

#define an epsilon greedy algorithm that will choose which action to take next (i.e., where to move next)
def get_next_action(current_row_index, current_column_index, epsilon, S1, S2, S3, S4):
    if np.random.random() > epsilon: #exploit
        return np.argmax(q_values[current_row_index, current_column_index])
    else: #explore
        #Obtain direction in terms of angle from the get_angle function
        angle = get_angle(S1, S2, S3, S4)
        #Transform the angle in direction in terms of direction index (Eg: 0=S, 1=W, 2=N, 3=E, .....)
        direction_index = actions.index(transform_angle(angle))
        return direction_index
                

#define a function that will get the next location based on the chosen action
def get_next_location(current_row_index, current_column_index, action_index):
    new_row_index = current_row_index
    new_column_index = current_column_index
    if new_column_index > environment_columns - 2:
        new_column_index -= 1
    elif new_row_index > environment_rows - 2:
        new_row_index -= 1
    else:
        if actions[action_index] == 'S' and current_row_index > 0:
            new_row_index -= 1
        elif actions[action_index] == 'E' and current_column_index < environment_columns - 1:
            new_column_index += 1
        elif actions[action_index] == 'N' and current_row_index < environment_rows - 1:
            new_row_index += 1
        elif actions[action_index] == 'W' and current_column_index > 0:
            new_column_index -= 1
        elif actions[action_index] == 'SE' and current_column_index > 0 and current_row_index > 0:
            new_row_index -= 1
            new_column_index += 1
        elif actions[action_index] == 'SW' and current_column_index > 0 and current_row_index > 0:
            new_row_index -= 1
            new_column_index -= 1
        elif actions[action_index] == 'NE' and current_column_index > 0 and environment_rows - 1:
            new_row_index += 1
            new_column_index += 1
        elif actions[action_index] == 'NW' and current_column_index > 0 and environment_rows - 1:
            new_row_index += 1
            new_column_index -= 1
    return new_row_index, new_column_index



#function for the drone to localize the source at the center
def get_drone_location(start_row_index, start_column_index, s1, s2, s3, s4):
    #return immediately if this is an invalid starting location (goal cell)
    if is_terminator(start_row_index, start_column_index):
        return []
    else: #if this is a valid starting location (not goal cell)
        current_row_index, current_column_index = start_row_index, start_column_index
        drone_location = []
        drone_location.append([current_row_index, current_column_index])
        #continue moving along the path until we reach the goal
        while not is_terminator(current_row_index, current_column_index):
            #get the best action to take
            #pass epsilon as 0 because this way the algorithm will always choose to exploit the env from the q-table
            action_index = get_next_action(current_row_index, current_column_index, 0, s1, s2, s3, s4)
            #move to the next location on the path, and add the new location to the list
            current_row_index, current_column_index = get_next_location(current_row_index, current_column_index, action_index)
            print(current_row_index, current_column_index)
            drone_location.append([current_row_index, current_column_index])
    return drone_location

In [11]:
#Just a plotting function
def plot(values, moving_avg_period):
    plt.figure(2)
    plt.clf()        
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(values)
    moving_avg = get_moving_average(moving_avg_period, values)
    plt.plot(moving_avg)
    print("Episode", len(values), "\n", \
        moving_avg_period, "episode moving avg:", moving_avg[-1])
    if is_ipython: display.clear_output(wait=True)

def get_moving_average(period, values):
    values = torch.tensor(values, dtype=torch.float)
    if len(values) >= period:
        moving_avg = values.unfold(dimension=0, size=period, step=1) \
            .mean(dim=1).flatten(start_dim=0)
        moving_avg = torch.cat((torch.zeros(period-1), moving_avg))
        return moving_avg.numpy()
    else:
        moving_avg = torch.zeros(len(values))
        return moving_avg.numpy()

Function to visualize the agent movement

In [12]:
def visualize_agent(path):
    # environment_rows = 17
    # environment_columns = 28

    env = [['.' for i in range(environment_columns)] for j in range(environment_rows)]
    env[8][15] = 0

    row=[]
    column=[]
    grid=[]
    for i in path:
        for j in i:
            grid.append(j)

    for m in range(len(grid)):
        if m%2 == 1:
            column.append(grid[m])
        else:
            row.append(grid[m])

    for i in range(len(row)):
        env[row[i]][column[i]] = '*'

    for r in env:
        for c in r:
            print(c,end = " ")
        print()
    time.sleep(.5)
    print("-----------------next movement--------------------")


In [13]:
ad.read_csv()

#define training parameters
# epsilon = [start, end, decay rate]
eps = [0.5, 0.01, 0.001]
#discount factor for future rewards
# epsilon = 0
discount_factor = 0.9
#the rate at which the AI agent should learn
learning_rate = 0.9

episodes = []
rewards_all_episodes = []
number_of_episodes = 29000
step = 0

for i in range(0,number_of_episodes):
    s0 = float(ad.S0_array[i])
    s1 = float(ad.S1_array[i])
    s2 = float(ad.S2_array[i])
    s3 = float(ad.S3_array[i])
    ep = 0
    step+=1
    agent_path = []
    rewards_current_episode = 0
    epsilon = EpsilonGreedyStrategy(eps, step)
    
    #get the starting location for this episode
    row_index, column_index = get_starting_location()
    
    #continue taking actions (i.e., moving) until we reach a terminal state
    while not is_terminal_state(row_index, column_index):
        #Function get_next_action() will return action_index (Eg: 0=S, 1=W, 2=N, 3=E, ...)
        action_index = get_next_action(row_index, column_index, epsilon, s0, s1, s2, s3)
        #perform the chosen action, and transition to the next state (i.e., move to the next location)
        old_row_index, old_column_index = row_index, column_index 
        agent_path.append([old_row_index, old_column_index])
        row_index, column_index = get_next_location(row_index, column_index, action_index)  
        if row_index > environment_rows:
            continue
        if column_index > environment_columns:
            continue

        #receive the reward for moving to the new state
        reward = rewards[row_index, column_index]
        #reward accumulation
        rewards_current_episode += reward
        old_q_value = q_values[old_row_index, old_column_index, action_index]
#         visualize_agent(agent_path)
        ep+=1
        #update the Q-value using bellman equation
        new_q_value = old_q_value + learning_rate*(reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value) 
        q_values[old_row_index, old_column_index, action_index] = new_q_value
    
    episodes.append(ep)     
    plot(episodes, 100)
#     print(agent_path)
#     visualize_agent(agent_path)
    rewards_all_episodes.append(rewards_current_episode)
    



print('Training complete!')
print(q_values)

# Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),number_of_episodes/1000)
count = 1000
ep=[]
print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    ep.append(sum(r/1000))
    count += 1000

#Plot the average reward accumulation for every 1000 episodes 
x=range(0,29)
print(ep)
plt.xlabel('number of episodes (x10^3)')
plt.ylabel('reward accumulated')
plt.plot(x, ep)


KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

Saving the Q-Table to a text file

In [ ]:
#reshaping the q-table from 3D matrix to 2D
qvalues = q_values.reshape(q_values.shape[0], -1)
  
#saving q-table to a file
np.savetxt("qvalues.txt", qvalues)